In [19]:
import time
from pathlib import Path

import cv2
import matplotlib.cm
import numpy as np
from IPython.display import display, clear_output, Audio
import openvino as ov
import ipywidgets as widgets
import os

from notebook_utils import download_file, load_image

# 모델 다운로드 및 로드
model_folder = Path("model")
ir_model_url_depth = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/depth-estimation-midas/FP32/"
ir_model_name_depth_xml = "MiDaS_small.xml"
ir_model_name_depth_bin = "MiDaS_small.bin"

ir_model_url_detection = "https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.4/models_bin/1/vehicle-detection-adas-0002/FP32/"
ir_model_name_detection_xml = "vehicle-detection-adas-0002.xml"
ir_model_name_detection_bin = "vehicle-detection-adas-0002.bin"

download_file(ir_model_url_depth + ir_model_name_depth_xml, filename=ir_model_name_depth_xml, directory=model_folder)
download_file(ir_model_url_depth + ir_model_name_depth_bin, filename=ir_model_name_depth_bin, directory=model_folder)
download_file(ir_model_url_detection + ir_model_name_detection_xml, filename=ir_model_name_detection_xml, directory=model_folder)
download_file(ir_model_url_detection + ir_model_name_detection_bin, filename=ir_model_name_detection_bin, directory=model_folder)

model_depth_xml_path = model_folder / ir_model_name_depth_xml
model_detection_xml_path = model_folder / ir_model_name_detection_xml

def normalize_minmax(data):
    """Normalizes the values in `data` between 0 and 1"""
    return (data - data.min()) / (data.max() - data.min())

def convert_result_to_image(result, colormap="viridis"):
    """
    Convert network result of floating point numbers to an RGB image with
    integer values from 0-255 by applying a colormap.

    `result` is expected to be a single network result in 1,H,W shape
    `colormap` is a matplotlib colormap.
    See https://matplotlib.org/stable/tutorials/colors/colormaps.html
    """
    cmap = matplotlib.cm.get_cmap(colormap)
    result = result.squeeze(0)
    result = normalize_minmax(result)
    result = cmap(result)[:, :, :3] * 255
    result = result.astype(np.uint8)
    return result

def to_rgb(image_data) -> np.ndarray:
    """Convert image_data from BGR to RGB"""
    return cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)

def play_warning_sound():
    """Play a warning sound"""
    duration = 1  # seconds
    freq = 440  # Hz
    os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

def sound(sound_file):
    wn = Audio(sound_file, autoplay=True)
    display(wn)


# OpenVINO Core 초기화 및 모델 컴파일
core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="AUTO",
    description="Device:",
    disabled=False,
)

# Create cache folder
cache_folder = Path("cache")
cache_folder.mkdir(exist_ok=True)

core.set_property({"CACHE_DIR": cache_folder})

# 깊이 맵 모델 컴파일
model_depth = core.read_model(model_depth_xml_path)
compiled_model_depth = core.compile_model(model=model_depth, device_name=device.value)

input_key_depth = compiled_model_depth.input(0)
output_key_depth = compiled_model_depth.output(0)

network_input_shape_depth = list(input_key_depth.shape)
network_image_height_depth, network_image_width_depth = network_input_shape_depth[2:]

# 차량 감지 모델 컴파일
model_detection = core.read_model(model_detection_xml_path)
compiled_model_detection = core.compile_model(model=model_detection, device_name=device.value)

input_key_detection = compiled_model_detection.input(0)
output_key_detection = compiled_model_detection.output(0)

network_input_shape_detection = list(input_key_detection.shape)
network_image_height_detection, network_image_width_detection = network_input_shape_detection[2:]

# 실시간 웹캠 피드와 거리 측정 및 차량 감지
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("/home/ubuntu/Videos/Screencasts/ch4.webm")
fps = 25  # 목표 프레임 속도 (FPS)
frame_time = 1.0 / fps  # 한 프레임의 시간 (초)


paused = False
paused_frame = None
alarm_triggered = False  # 경고음을 재생했는지 추적하는 변수
frame_count = 0
consecutive_depth_count = 0  # 연속된 깊이 값이 200을 초과하는 경우의 횟수를 추적하는 변수

while cap.isOpened():
    start_time = time.time()
    if not paused:
        ret, frame = cap.read()
        if not ret:
            break
        # 차량 감지
        resized_image_detection = cv2.resize(src=frame, dsize=(network_image_width_detection, network_image_height_detection))
        input_image_detection = np.expand_dims(np.transpose(resized_image_detection, (2, 0, 1)), 0)
        result_detection = compiled_model_detection([input_image_detection])[output_key_detection]
        # 화면 가운데 좌표 (중앙선 위치)
        center_x_line = frame.shape[1]*0.6

        # 목록으로 깊이 값 저장
        depth_values = []
        
        # 차량 감지 결과 표시
        for detection in result_detection[0][0]:
            confidence = detection[2]
            if confidence > 0.5:  # 감지 신뢰도가 50% 이상인 경우
                xmin = int(detection[3] * frame.shape[1])
                ymin = int(detection[4] * frame.shape[0])
                xmax = int(detection[5] * frame.shape[1])
                ymax = int(detection[6] * frame.shape[0])
                
                # 감지된 차량 영역의 중심점 계산
                center_x = (xmin + xmax) // 2
                center_y = (ymin + ymax) // 2
                
                if center_x < center_x_line:
                    cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                    #cv2.putText(frame, f'Confidence: {confidence:.2f}', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    # 중심점 좌표를 이용하여 깊이(거리) 추정
                    resized_image_depth = cv2.resize(src=frame, dsize=(network_image_width_depth, network_image_height_depth))
                    input_image_depth = np.expand_dims(np.transpose(resized_image_depth, (2, 0, 1)), 0)
                    result_depth = compiled_model_depth([input_image_depth])[output_key_depth]
    
                    # 결과 이미지를 컬러맵으로 변환
                    result_image = convert_result_to_image(result=result_depth)
                    
                    # 원본 이미지 크기로 다시 조정
                    result_image = cv2.resize(result_image, frame.shape[:2][::-1])
                    
                    result_depth_rescaled = cv2.resize(result_depth[0], (frame.shape[1], frame.shape[0]))
                    
                    depth_value = result_depth_rescaled[center_y, center_x]  # 수정된 부분
                    cv2.putText(frame, f'Distance: {depth_value:.2f}', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    print(f"Detected vehicle depth at center: {depth_value}")
                    depth_values.append(depth_value)  # 깊이 값을 리스트에 추가
        
        # 리스트에서 최대 깊이 값 찾기
        if depth_values:
            max_depth_value = max(depth_values)
            print(f"Maximum depth value among detected vehicles: {max_depth_value}")
            if max_depth_value > 200.0:
                consecutive_depth_count += 1
                if consecutive_depth_count >= 5:
                    cv2.putText(frame, 'Danger!!', (frame.shape[1] // 4, frame.shape[0] // 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    if not alarm_triggered:  # 경고음이 아직 재생되지 않은 경우에만 재생
                        sound("./danger.mp3")
                        alarm_triggered = True
            else:
                alarm_triggered = False  # 조건이 만족되지 않으면 경고음 상태를 초기화
                consecutive_depth_count = 0 
        else:
            print("No vehicles detected.")
            alarm_triggered = False  # 차량이 감지되지 않으면 경고음 상태를 초기화
            consecutive_depth_count = 0 

        # 프레임을 일시정지 상태에서 사용할 수 있도록 저장
        paused_frame = frame.copy()
        
    else:
        # 일시정지 상태에서는 저장된 프레임을 표시
        if paused_frame is not None:
            frame = paused_frame
    
    # combined_image = np.hstack((frame, result_image))
    cv2.imshow('Object Detection and Depth Estimation', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('p'):
        paused = not paused
        if paused:
            print("Paused")
        else:
            print("Resumed")

    end_time = time.time()
    elapsed_time = end_time - start_time
    if elapsed_time < frame_time:
        time.sleep(frame_time - elapsed_time)

cap.release()
cv2.destroyAllWindows()

'model/MiDaS_small.xml' already exists.
'model/MiDaS_small.bin' already exists.
'model/vehicle-detection-adas-0002.xml' already exists.
'model/vehicle-detection-adas-0002.bin' already exists.
No vehicles detected.
No vehicles detected.
No vehicles detected.
No vehicles detected.
No vehicles detected.
No vehicles detected.
Detected vehicle depth at center: 120.12142181396484
Maximum depth value among detected vehicles: 120.12142181396484
Detected vehicle depth at center: 120.80329132080078
Maximum depth value among detected vehicles: 120.80329132080078
Detected vehicle depth at center: 121.67041778564453
Maximum depth value among detected vehicles: 121.67041778564453
Detected vehicle depth at center: 120.62772369384766
Maximum depth value among detected vehicles: 120.62772369384766
Detected vehicle depth at center: 121.50696563720703
Maximum depth value among detected vehicles: 121.50696563720703
Detected vehicle depth at center: 123.7304458618164
Maximum depth value among detected vehi

Detected vehicle depth at center: 206.07305908203125
Detected vehicle depth at center: 152.9791259765625
Maximum depth value among detected vehicles: 206.07305908203125
Detected vehicle depth at center: 268.85003662109375
Maximum depth value among detected vehicles: 268.85003662109375
Detected vehicle depth at center: 289.6913757324219
Detected vehicle depth at center: 161.19638061523438
Maximum depth value among detected vehicles: 289.6913757324219
Detected vehicle depth at center: 254.72314453125
Detected vehicle depth at center: 163.56625366210938
Maximum depth value among detected vehicles: 254.72314453125
Detected vehicle depth at center: 273.6822509765625
Detected vehicle depth at center: 159.28468322753906
Maximum depth value among detected vehicles: 273.6822509765625
Detected vehicle depth at center: 270.0695495605469
Detected vehicle depth at center: 162.67520141601562
Maximum depth value among detected vehicles: 270.0695495605469
Detected vehicle depth at center: 271.86645507

Detected vehicle depth at center: 313.5518493652344
Maximum depth value among detected vehicles: 313.5518493652344
Detected vehicle depth at center: 336.22021484375
Maximum depth value among detected vehicles: 336.22021484375
Detected vehicle depth at center: 330.8130798339844
Maximum depth value among detected vehicles: 330.8130798339844
Detected vehicle depth at center: 292.75347900390625
Maximum depth value among detected vehicles: 292.75347900390625
Detected vehicle depth at center: 332.8233337402344
Maximum depth value among detected vehicles: 332.8233337402344
Detected vehicle depth at center: 310.9016418457031
Maximum depth value among detected vehicles: 310.9016418457031
Detected vehicle depth at center: 323.08447265625
Maximum depth value among detected vehicles: 323.08447265625
Detected vehicle depth at center: 341.986328125
Maximum depth value among detected vehicles: 341.986328125
Detected vehicle depth at center: 326.541259765625
Maximum depth value among detected vehicles

Detected vehicle depth at center: 263.00799560546875
Detected vehicle depth at center: 203.2947998046875
Maximum depth value among detected vehicles: 263.00799560546875
Detected vehicle depth at center: 284.44537353515625
Detected vehicle depth at center: 205.72572326660156
Maximum depth value among detected vehicles: 284.44537353515625
Detected vehicle depth at center: 232.3745880126953
Detected vehicle depth at center: 190.77186584472656
Maximum depth value among detected vehicles: 232.3745880126953
Detected vehicle depth at center: 291.29583740234375
Detected vehicle depth at center: 199.1766815185547
Maximum depth value among detected vehicles: 291.29583740234375
Detected vehicle depth at center: 269.5325927734375
Detected vehicle depth at center: 190.6381072998047
Maximum depth value among detected vehicles: 269.5325927734375
Detected vehicle depth at center: 318.0549621582031
Detected vehicle depth at center: 191.24209594726562
Maximum depth value among detected vehicles: 318.054

Detected vehicle depth at center: 267.2296447753906
Maximum depth value among detected vehicles: 267.2296447753906
Detected vehicle depth at center: 287.3603210449219
Maximum depth value among detected vehicles: 287.3603210449219
Detected vehicle depth at center: 341.87060546875
Maximum depth value among detected vehicles: 341.87060546875
Detected vehicle depth at center: 338.00341796875
Maximum depth value among detected vehicles: 338.00341796875
Detected vehicle depth at center: 314.6009826660156
Maximum depth value among detected vehicles: 314.6009826660156
Detected vehicle depth at center: 334.7401428222656
Maximum depth value among detected vehicles: 334.7401428222656
Detected vehicle depth at center: 337.885009765625
Maximum depth value among detected vehicles: 337.885009765625
Detected vehicle depth at center: 354.6770324707031
Maximum depth value among detected vehicles: 354.6770324707031
Detected vehicle depth at center: 382.4610595703125
Maximum depth value among detected veh

In [8]:
play_warning_sound()

sh: 1: play: not found
